In [1]:
import sys
sys.path.append('/home/ego/Github/david/')

from sys import stdout
import datetime
import dataset

from david.youtube import scraper

FILE = 'pending_downloads.txt'
DB_SQL = 'sqlite:///yt_comments.db'
DB_TABLE = 'comments'

today = datetime.datetime.now()
#today = '{:%Y-%m-%d %H:%M:%S}'.format(today)

db = dataset.connect(DB_SQL)
table = db.create_table(DB_TABLE)
#table.create_column('timestamp', db.types.datetime)

In [2]:
def get_videos(fp):
    with open(fp, 'r', encoding='utf-8') as f:
        videos = []
        for line in f.readlines():
            videos.append(line.strip('\n'))
        return videos

def download_comments(videoid: str, table=DB_TABLE, sqlite_url=DB_SQL):
    count = 0
    with dataset.connect(sqlite_url) as sqlite:
        for comment in scraper._scrape_comments(videoid):
            sqlite[table].insert(dict(
                cid=comment['cid'], text=comment['text'],
                time=comment['time'], author=comment['author'],
                video_id=videoid)
            )
            count += 1
            stdout.write('mining %d comment(s)\r' % count)
            stdout.flush()

def batch_downloader(fp, table, sqlite_url):
    video_ids = get_videos(fp)
    for video in video_ids:
        download_comments(videoid=video)

In [3]:
batch_downloader(FILE, DB_TABLE, DB_SQL)

/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/dataset/table.py:218: RuntimeWarning: Changing the database schema inside a transaction in a multi-threaded environment is likely to lead to race conditions and synchronization issues.
  RuntimeWarning)


Exception during reset or similar
Traceback (most recent call last):
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 867, in _reset
    pool._dialect.do_rollback(self)
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/engine/default.py", line 500, in do_rollback
    dbapi_connection.rollback()
sqlite3.ProgrammingError: SQLite objects created in a thread can only be used in that same thread. The object was created in thread id 139757836130112 and this is thread id 139757758199552.
Exception closing connection <sqlite3.Connection object at 0x7f1bc95ddd50>
Traceback (most recent call last):
  File "/home/ego/anaconda3/envs/vuepoint/lib/python3.6/site-packages/sqlalchemy/pool/base.py", line 680, in _finalize_fairy
    fairy._reset(pool)
  File "/home/ego/anacond

ParserError: Document is empty

In [6]:
table.count()

252848

In [22]:
statement = 'SELECT DISTINCT video_id FROM comments;'
for row in db.query(statement):
    print(row['video_id'])

2FmcHiLCwTU
bhxhNIQBKJI
f9573kGBtuE
XnbCSboujF4
ad_higXixRA
gf_MqDBBMPI
EcojyBV4QJ4
R2pIutTspQA
XOLOLrUBRBY
seJSm-vD9OA
pU9iJB0ATpk
voYuH5eGlkk
6DasnKpMyQ8
u4ZoJKF_VuA


In [35]:
statement = 'SELECT video_id, COUNT(*) c FROM comments GROUP BY video_id;'

print('video_id  | comment count \n')

total_comments = 0
for num, row in enumerate(db.query(statement)):
    print(f"{num}: {row['video_id']} -> {row['c']}")
    total_comments += row['c']

print(f'\ntotal comments in database: {total_comments}')

video_id  | comment count 

0: 2FmcHiLCwTU -> 1085
1: 6DasnKpMyQ8 -> 18723
2: EcojyBV4QJ4 -> 3686
3: R2pIutTspQA -> 3577
4: XOLOLrUBRBY -> 7643
5: XnbCSboujF4 -> 25783
6: ad_higXixRA -> 14882
7: bhxhNIQBKJI -> 59862
8: f9573kGBtuE -> 93889
9: gf_MqDBBMPI -> 8354
10: pU9iJB0ATpk -> 7253
11: seJSm-vD9OA -> 2138
12: u4ZoJKF_VuA -> 1040
13: voYuH5eGlkk -> 4933

total comments in database: 252848
